<div style="color:#800">Attention le rendu de ce notebook sur Github est buggé : les balises XML sont interprétées au lieu d'être représentées. Le fichier .ipynb sur le dépôt Github est donc uniquement destiné à être téléchargé, pas visualisé sur place. Pour une version HTML, utiliser l'adresse :
http://dmolinarius.github.io/demofiles/mod-32/07_NoSQL//05_BaseX.html
</div>

## Base de données XML : BaseX

<div style="font-size:120%">
Installation cf. https://docs.basex.org/wiki/Startup via le package adéquat
</div>
<div style="font-size:120%; margin-top:1em">
&rightarrow; fichier .jar ou installeur Windows sur https://basex.org/download/
</div>
<div style="font-size:120%; margin-top:1em">
Démarrage du serveur HTTP <span style="font-family:Consolas; font-size:90%">~BaseX\bin\basexhttpstop.bat</span> (par défaut sur le port 8080)
</div>


In [1]:
import http.client

with open('__baseX_credentials.txt', 'r') as f:
    credentials = f.read().strip()

def baseX_request(method, path, body=None, headers={}):
    conn = http.client.HTTPConnection('127.0.0.1', 8080)
    headers['Authorization'] = 'Basic {}'.format(credentials)
    conn.request(method, path, body.encode('UTF-8'), headers)
    response = conn.getresponse()
    body = response.read()
    return body.decode()

<div id="create-DB"></div>
<h3>Création d'une base de données</h3>
<p style="font-size:120%">Document source :</p>

In [2]:
document = '''
<people>
 <person>
   <first_name>Raymond</first_name>
   <last_name>Deubaze</last_name>
   <email>raymond.deubaze@ec-lyon.fr</email>
 </person>
 <person>
   <first_name>Jean</first_name>
   <last_name>Peuplu</last_name>
   <email>jpu@gmail.com</email>
 </person>
 <person>
   <first_name>Alex</first_name>
   <last_name>Terrieur</last_name>
   <email>atr1@gmail.com</email>
 </person>
 <person>
   <first_name>Alain</first_name>
   <last_name>Terrieur</last_name>
   <email>atr2@gmail.com</email>
 </person>
  <person>
   <first_name>Anna</first_name>
   <last_name>Conda</last_name>
   <email>Anna.Conda@ec-lyon.fr</email>
 </person>
 <person>
   <first_name>Ginette</first_name>
   <last_name>Ringard</last_name>
   <email>ginette@wanadoo.fr</email>
 </person>
</people>'''

<div style="font-size:120%">
Création et alimentation de la base.
</div>
<p style="font-size:120%">Si la base de données existe déjà, elle sera écrasée...</p>

In [4]:
body = '''
<commands>
  <create-db name='people'>{}</create-db>
  <list/>
</commands>
'''.format(document)

resp = baseX_request('POST', '/rest', body)
print(resp)

Name    Resources  Size  Input Path  
-----------------------------------
people  1          5660  people.xml  

1 database(s).



<h3>Requêtes XQuery</h3>
<p id="query-XPath" style="font-size:120%">Expression XPath</p>

In [4]:
# Requête XPath
body = '''
<query>
  <text>doc('people')//person[last_name="Terrieur"]/email/text()</text>
</query>
'''

resp = baseX_request('POST', '/rest', body)
print(resp)

atr1@gmail.com
atr2@gmail.com


In [8]:
body = '''
<query>
  <text>
    doc('people')//person[
      contains(email,"gmail.com") and
      not(starts-with(first_name,'A'))
    ]
  </text>
</query>
'''

resp = baseX_request('POST', '/rest', body)
print(resp)

<person>
   <first_name>Jean</first_name>
   <last_name>Peuplu</last_name>
   <email>jpu@gmail.com</email>
 </person>


<div id="query-XQuery" style="font-size:120%">Expression XQuery</div>

In [5]:
# Requête XQuery
body = '''
<query>
  <text>
    for $person in doc("people")//person
      where contains($person/email,'ec-lyon')
      return $person/email/text()
  </text>
</query>
'''

resp = baseX_request('POST', '/rest', body)
print(resp)

raymond.deubaze@ec-lyon.fr
Anna.Conda@ec-lyon.fr


<div id="query-FLWOR" style="font-size:120%">Expression FLWOR</div>

In [6]:
# Requête FLWOR
body = '''
<query>
  <text>
    for $person in doc("people")//person
      let $email := $person/email
      where contains($email,'wanadoo')
      order by $person/last_name
      return $person
  </text>
</query>
'''

resp = baseX_request('POST', '/rest', body)
print(resp)

<person>
   <first_name>Ginette</first_name>
   <last_name>Ringard</last_name>
   <email>ginette@wanadoo.fr</email>
 </person>


<div id="generate-XML"></div>
<h3>Génération de code HTML ou XML</h3>

In [7]:
body = '''
<query>
  <text><![CDATA[
    <ul>
    {for $person in doc("people")//person
      let $email := $person/email
      where contains($email,'ec-lyon.fr')
      order by $person/last_name
      return <li>{$person/first_name/text()}&#160;{$person/last_name/text()}</li>
    }</ul>
  ]]></text>
</query>
'''

resp = baseX_request('POST', '/rest', body)
print(resp)

<ul><li>Anna Conda</li><li>Raymond Deubaze</li></ul>


<div id="predefined-function"></div>
<h3>Fonctions prédéfinies</h3>
<p>https://www.w3.org/TR/xpath-functions-31/</p>

In [8]:
body = '''
<query>
  <text>
    math:pi()
  </text>
</query>
'''

resp = baseX_request('POST', '/rest', body)
print(resp)

3.141592653589793


<div id="user-function"></div>
<h3>Fonction utilisateur</h3>

In [9]:
body = '''
<query>
  <text>
    declare function local:full_name($p as element(person)) as xs:string
    {
      concat($p/first_name,' ',$p/last_name)
    };
  <![CDATA[
    <ul>{
    for $person in doc("people")//person
      let $email := $person/email
      where contains($email,'ec-lyon.fr')
      order by $person/last_name
      return <li>{local:full_name($person)}</li>
    }</ul>
  ]]></text>
</query>
'''

resp = baseX_request('POST', '/rest', body)
print(resp)

<ul><li>Anna Conda</li><li>Raymond Deubaze</li></ul>


<div id="Newton" style="font-size:120%">
Calcul de racine carrée par la méthode de Newton
</div>
<p>
https://files.basex.org/.xml15/slides/05-XQuery-II.pdf
</p>

In [10]:
body = '''
<query>
  <text>
    declare function local:square-root($n as xs:double, $eps as xs:double) {
      local:square-root($n, 1, $eps)
    };
    declare function local:square-root($n, $guess, $eps) {
      let $new:= $n div ($guess)
      let $diff:= abs($n - $guess * $guess)
      return if ($diff lt $eps) then $guess else
      let $better-guess := ($guess + $new) div 2
      return local:square-root($n, $better-guess, $eps)
    };
    local:square-root(2, 0.000001)
</text>
</query>
'''

resp = baseX_request('POST', '/rest', body)
print(resp)

1.4142135623746899
